# Pygamon

Après l'installation des différentes bibliothèque pour l'utilisation (pygame, pyscroll, pytmx) nous allons commencer par crée la boucle principale de projet 

___

## Run de la fenêtre

voici le code de base pour faire tournée une fenêtre pygame

In [2]:
import pygame
pygame.init()

#création de la fenêtre
pygame.display.set_mode((800, 600))
pygame.display.set_caption("Pygamon - Aventure")

# boucle du jeu
running = True

while running :
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
pygame.quit()

## Création de la carte 

Nous allons crée les cartes sur Tiled. Pas besoin de plus de détail pour cette étape


____


## Rangement dans des classes et des fichier

Commencer un rangement de notre code en class. Ranger dans deux fichier distint

###### main.py

In [ ]:
import pygame

from game import Game

if __name__ == '__main__' :
    pygame.init()
    game = Game()
    game.run()

###### game.py

In [ ]:
import pygame

class Game:
    def __init__(self):
        #création de la fenêtre
        pygame.display.set_mode((800, 600))
        pygame.display.set_caption("Pygamon - Aventure")
        
    def run(self):
        # boucle du jeu
        running = True

        while running :
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

        pygame.quit()

## Intégration de la carte 

Je vais maintenant intégrer la carte à notre fenêtre de jeu.

###### main.py

pas de changement

###### game.py

In [ ]:
import pygame
import pytmx #module pour intégrer la carte
import pyscroll #module pour la gestion de la caméra

class Game:
    def __init__(self):
        #création de la fenêtre
        self.screen = pygame.display.set_mode((800, 600)) #stockage dans une variable
        pygame.display.set_caption("Pygamon - Aventure")
        
        #charger la carte (tmx)
        tmx_data = pytmx.util_pygame.load_pygame('carte.tmx')# Aborder le rangement hiérachique des fichier
        map_data = pyscroll.data.TiledMapData(tmx_data) # traduction en map
        map_layer = pyscroll.orthographic.BufferRenderer(map_data, self.screen.get_size())
        
        # zome sur la carte
        map_layer.zoom = 2 # à rajouter à la fin
        
        #dessiner le groupe de calques 
        self.group = pyscroll.PyscrollGroup(map_layer=map_layer, default_layer=1)
    
    def run(self):
        # boucle du jeu
        running = True

        while running :
            
            # affichage de la carte 
            self.group.draw(self.screen)
            
            pygame.display.flip() #mettre à jour tout
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

        pygame.quit()

### Ordre de passage 

#### 1) Stockage du screen 
    self.screen = pygame.display.set_mode((800, 600))
Stockage de la fenêtre dans une variable pour pouvoir la localiser et l'utiliser

#### 2) Charger la carte (tmx)
      tmx_data = pytmx.util_pygame.load_pygame('carte.tmx')
      map_data = pyscroll.data.TiledMapData(tmx_data) 
      map_layer = pyscroll.orthographic.BufferRenderer(map_data, self.screen.get_size())
Faire l'architecture du projet charger la première carte et faire en sorte de pouvoir gerer l'affichage technique de la map

#### 3) Dessiner le groupe de calques 
    self.group = pyscroll.PyscrollGroup(map_layer=map_layer, default_layer=1)
Crée un groupe avec pyscroll pour permettre l'affichage plus tard

#### 4) Affichage de la carte 
    self.group.draw(self.screen)
            
    pygame.display.flip() #mettre à jour tout

ajout de c'est ligne dans la boucle principale pour permettre l'affichage de la carte et la mise à jour de pygame

#### 5) Run de démonstration

faire un run de démonstration pour montrer le résultas

#### 6) Zome sur la carte
    map_layer.zoom = 2 
Ajout de cette ligne pour créer un zoom sur la carte

## Création du joueur

cette partie va intégrer tout le principe de création du joueur: intégration sur la map et affichage

###### main.py
Pas de changement

###### player.py

In [ ]:
import pygame

class Player(pygame.sprite.Sprite):
    def __init__(self, x , y):
        super().__init__()
        self.sprite_sheet = pygame.image.load('player.png')
        self.image = self.get_image(0, 0)
        self.image.set_colorkey([0, 0, 0])
        self.rect = self.image.get_rect()
        self.position = [x, y]
        
    def update(self):
        self.rect.topleft = self.position
        
    def get_image(self, x, y):
        image = pygame.Surface([32, 32])
        image.blit(self.sprite_sheet, (0, 0), (x, y, 32, 32))
        return image

###### game.py

In [ ]:
import pygame
import pytmx # module pour intégrer la carte
import pyscroll # module pour la gestion de la caméra

from player import Player

class Game:
    def __init__(self):
        # création de la fenêtre
        self.screen = pygame.display.set_mode((800, 600)) # stockage dans une variable
        pygame.display.set_caption("Pygamon - Aventure")
        
        # charger la carte (tmx)
        tmx_data = pytmx.util_pygame.load_pygame('carte.tmx')# Aborder le rangement hiérachique des fichier
        map_data = pyscroll.data.TiledMapData(tmx_data) # traduction en map
        map_layer = pyscroll.orthographic.BufferRenderer(map_data, self.screen.get_size())
        
        # zome sur la carte
        map_layer.zoom = 2 
        
        # generer un joueur
        player_position = tmx_data.get_object_by_name("player")
        self.player = Player(player_position.x , player_position.y)
        
        #dessiner le groupe de calques 
        self.group = pyscroll.PyscrollGroup(map_layer=map_layer, default_layer=3) # calque changement
        self.group.add(self.player)
    
    def run(self):
        # boucle du jeu
        running = True

        while running :
            
            # mettre à jour la position du joueur 
            self.group.update()
            
            # centrer la caméra sur le joueur
            self.group.center(self.player.rect.center)
            
            # affichage de la carte 
            self.group.draw(self.screen)
            
            pygame.display.flip() #mettre à jour tout
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

        pygame.quit()

### Ordre de passage

#### 1)  Ajout de l'image au projet
parler de l'ajout de l'image du joueur de son rangement et de la création d'un script rien que pour la gestion du joueur

#### 2) Créattion de la class Player
__player.py__

    import pygame

    class Player(pygame.sprite.Sprite):
        def __init__(self):
            super().__init__()
            self.sprite_sheet = pygame.image.load('player.png')
        
        def get_image(self, x, y):
            image = pygame.Surface([32, 32])
            image.blit(self.sprite_sheet, (0, 0), (x, y, 32, 32))
            return image
création de la classe player qui hérite de Sprite de pygame (explication sur qu'est ce qu'un sprite et la diférence entre le sprite et le background) explication à propos du __super().--init--()__. Chargement de l'image et stockage dans une variable. création de get_image d'abord sans paramètre puis ajout des paramètre explication de chaque ligne ètape par étape (découpage 32 par 32, blit() cadrage de la partie de l'image retenue, return retour de l'image)

#### 3) Application de get_image et rectangle d'image
__player.py__

    import pygame

    class Player(pygame.sprite.Sprite):
        def __init__(self):
            super().__init__()
            self.sprite_sheet = pygame.image.load('player.png')
    ---->   self.image = self.get_image(0, 0)
    ---->   self.rect = self.image.get_rect()
              
        def get_image(self, x, y):
            image = pygame.Surface([32, 32])
            image.blit(self.sprite_sheet, (0, 0), (x, y, 32, 32))
            return image
Utilisation de la function get_image() et stockage dans une variable. création du rectangle de collision

#### 4) Gération du joueur 
__game.py__ 

    from player import Player
    
    self.player = Player()
    
imporation de la class player dans game et stockage dans une variable

    self.group.add(self.player)
ajout dans la variable group qui gére l'affichage 


__Lancer un teste à la fin de cette étape pour démontrer__

#### 5) Supression du fond noir

__player.py__

    class Player(pygame.sprite.Sprite):
        def __init__(self):
            super().__init__()
            self.sprite_sheet = pygame.image.load('player.png')
            self.image = self.get_image(0, 0)
    ---->   self.image.set_colorkey([0, 0, 0])
            self.rect = self.image.get_rect()

supression du fond noir avec une ligne de code dans player.py

__Lancer un teste à la fin de cette étape__

#### 6) repositioner le joueur avec des coordonée
__player.py__

    class Player(pygame.sprite.Sprite):
    ---->   def __init__(self, x, y):
                super().__init__()
                self.sprite_sheet = pygame.image.load('player.png')
                self.image = self.get_image(0, 0)
                self.image.set_colorkey([0, 0, 0])
                self.rect = self.image.get_rect()
    ---->       self.position = [x, y]
        
    ---->   def update(self):
    ---->       self.rect.topleft = self.position

Ajout de paramettre à init de player. Ajout d'une ligne utilisant ses paramètres. Création d'une nouvelle fonction qui mettra àjour la position du joueur dans la boucle.

__game.py__
    
    self.player = Player(40, 30)
    
    self.group.update()
    
changement des paramètre de la déclaration de player. et ajout d'une ligne pour mettre à jour la position du joueur dan sla boucle principale

__Lancer teste et changer la position plusieur fois pour démonter__

#### 7) Création d'objet dans Tiled
Création d'un calque objet dans tiled et ajout d'un point de spawn appelé player

implémentation dans le code :

__game.py__

    player_position = tmx_data.get_object_by_name("player")
    self.player = Player(player_position.x, player_position.y)
    
Ajout d'une variable qui récupère l'objet spawn qui s'appeles player. Changement des paramètre de Player par la position de l'objet spawn

__lancer test__

#### 8) Recentrer la camèra sur le joueur
__game.py___

    self.group.center(self.player.rect.center)
Ligne ajouter dans run en dessou de l'update de la position du joueur pour recenter la camèra sur le joueur 

__lancer test et faire exprès de palcer le joueur sous un calque__

#### 9) Chagement du niveau du calque 
__game.py__

    self.group = pyscroll.PyscrollGroup(map_layer=map_layer, default_layer=3)

changement du niveau du claque sur trois pour passer le joueur au dessus de tout les calques

__Lancer test__


## Deplacement du joueur

nous allons nous occupé du déplacement du joueur et de changer ça direction quand il change de direction

###### main.py

pas de changement

###### game.py

In [ ]:
import pygame
import pytmx # module pour intégrer la carte
import pyscroll # module pour la gestion de la caméra

from player import Player

class Game:
    def __init__(self):
        # création de la fenêtre
        self.screen = pygame.display.set_mode((800, 600)) # stockage dans une variable
        pygame.display.set_caption("Pygamon - Aventure")
        
        # charger la carte (tmx)
        tmx_data = pytmx.util_pygame.load_pygame('carte.tmx')# Aborder le rangement hiérachique des fichier
        map_data = pyscroll.data.TiledMapData(tmx_data) # traduction en map
        map_layer = pyscroll.orthographic.BufferRenderer(map_data, self.screen.get_size())
        
        # zome sur la carte
        map_layer.zoom = 2 
        
        # generer un joueur
        player_position = tmx_data.get_object_by_name("player")
        self.player = Player(player_position.x , player_position.y)
        
        #dessiner le groupe de calques 
        self.group = pyscroll.PyscrollGroup(map_layer=map_layer, default_layer=3) # calque changement
        self.group.add(self.player)
        
    def handle_input(self):
        pressed = pygame.key.get_pressed()
        
        if pressed[pygame.K_UP]:
            self.player.move_up()
            self.player.change_animation('up')
        elif pressed[pygame.K_DOWN]:
            self.player.move_down()
            self.player.change_animation('down')
        elif pressed[pygame.K_LEFT]:
            self.player.move_left()
            self.player.change_animation('left')
        elif pressed[pygame.K_RIGHT]:
            self.player.move_right()
            self.player.change_animation('right')
    
    def run(self):
        #mise en place des fps
        clock = pygame.time.Clock()
        
        # boucle du jeu
        running = True

        while running :
            
            # récupération des touches
            self.handle_input()
            
            # mettre à jour la position du joueur 
            self.group.update()
            
            # centrer la caméra sur le joueur
            self.group.center(self.player.rect.center)
            
            # affichage de la carte 
            self.group.draw(self.screen)
            
            pygame.display.flip() #mettre à jour tout
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
            
            clock.tick(60)

        pygame.quit()

###### player.py

In [ ]:
import pygame

class Player(pygame.sprite.Sprite):
    def __init__(self, x , y):
        super().__init__()
        self.sprite_sheet = pygame.image.load('player.png')
        self.image = self.get_image(0, 0)
        self.image.set_colorkey([0, 0, 0])
        self.rect = self.image.get_rect()
        self.position = [x, y]
        self.images = {
            'down': self.get_image(0, 0),
            'left': self.get_image(0, 32),
            'right': self.get_image(0, 64,
            'up': self.get_image(0, 96)
        }
        self.speed = 3
    
    def change_animation(self, name):                             
        self.image = self.images[name]
        self.image.set_colorkey([0, 0, 0])
        
    def move_right(self): self.position[0] += self.speed
        
    def move_left(self): self.position[0] -= self.speed
        
    def move_up(self): self.position[1] += self.speed
        
    def move_down(self): self.position[1] -= self.speed
        
    def update(self):
        self.rect.topleft = self.position
        
    def get_image(self, x, y):
        image = pygame.Surface([32, 32])
        image.blit(self.sprite_sheet, (0, 0), (x, y, 32, 32))
        return image

### Ordre de passage 
#### 1) Ajout d'une fonction de détection de touche 
nous allons crée une nouvelle fonction nommé __handle_input__ qui gérera la détection de touche et ensuite l'appeler dans la boucle principale du jeu

    def handle_input(self):
        pressed = pygame.key.get_pressed()
        
        if pressed[pygame.K_UP]:
            print("haut")
        elif pressed[pygame.K_DOWN]:
            print("bas")
        elif pressed[pygame.K_LEFT]:
            print("gauche")
        elif pressed[pygame.K_RIGHT]:
            print("droite")
            
intégration dans la boucle principale

    while running :
    ---->    # récupération des touches
    ---->    self.handle_input()
            
              # mettre à jour la position du joueur 
              self.group.update()
            
              # centrer la caméra sur le joueur
              self.group.center(self.player.rect.center)
            
              # affichage de la carte 
              self.group.draw(self.screen)
            
              pygame.display.flip() #mettre à jour tout
            
              for event in pygame.event.get():
                  if event.type == pygame.QUIT:
                      running = False
                      
__lancer un teste___
#### 2) Mise en place du déplacement 
nous allons vraiment déplacer notre sprite de joueur en ajoutant des fonction de déplacemnt et en les utilisant lors de la détection d'un touche

__player.py__

    def move_right(self): self.position[0] += self.speed
        
    def move_left(self): self.position[0] -= self.speed
        
    def move_up(self): self.position[1] += self.speed
        
    def move_down(self): self.position[1] -= self.speed
    
__game.py__

    def handle_input(self):
            pressed = pygame.key.get_pressed()
        
            if pressed[pygame.K_UP]:
    ---->       self.player.move_up()
            elif pressed[pygame.K_DOWN]:
    ---->       self.player.move_down()
            elif pressed[pygame.K_LEFT]:
    ---->       self.player.move_left()
            elif pressed[pygame.K_RIGHT]:
    ---->        self.player.move_right()
    
    
__lancer un teste__

#### 3) Bridage des fps 
le jeu tourne beaucoup trop vite voila pourquoi il va nous falloir brider ça vitesse grece à quelque ligne 


    def run(self):
    ---->   #mise en place des fps
    ---->   clock = pygame.time.Clock()

            # boucle du jeu
            running = True

            while running :

                # récupération des touches
                self.handle_input()

                # mettre à jour la position du joueur 
                self.group.update()

                # centrer la caméra sur le joueur
                self.group.center(self.player.rect.center)

                # affichage de la carte 
                self.group.draw(self.screen)

                pygame.display.flip() #mettre à jour tout

                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False

    ---->       clock.tick(60)

            pygame.quit()
            
__lancer un teste__

(possibiliter de changer la vitesse du joueur)

#### 4)  animation de déplacement basique 
on va ajouter un découpage de l'image dans un dictionnaire trié par direction et ensuite crée une fonction pour modifier l'image

__player.py__

    class Player(pygame.sprite.Sprite):
            def __init__(self, x , y):
                super().__init__()
                self.sprite_sheet = pygame.image.load('player.png')
                self.image = self.get_image(0, 0)
                self.image.set_colorkey([0, 0, 0])
                self.rect = self.image.get_rect()
                self.position = [x, y]
    ---->       self.images = {
    ---->           'down': self.get_image(0, 0),
    ---->           'left': self.get_image(0, 32),
    ---->           'right': self.get_image(0, 64,
    ---->           'up': self.get_image(0, 96)
    ---->       }
                self.speed = 3

    ---->    def change_animation(self, name): self.image = self.images[name]
    
__game.py__

    def handle_input(self):
            pressed = pygame.key.get_pressed()
        
            if pressed[pygame.K_UP]:
                self.player.move_up()
    ---->       self.player.change_animation('up')
            elif pressed[pygame.K_DOWN]:
                self.player.move_down()
    ---->       self.player.change_animation('down')
            elif pressed[pygame.K_LEFT]:
                 self.player.move_left()
    ---->        self.player.change_animation('left')
            elif pressed[pygame.K_RIGHT]:
                 self.player.move_right()
    ---->        self.player.change_animation('right')
    
    
__lancer un teste___ (expliqué la remise en place du fond noire)

    def change_animation(self, name):                             
                self.image = self.images[name]
    ---->       self.image.set_colorkey([0, 0, 0])
    
__lancer un teste__


## Gestion des collision 
Nous allons mettre en place les collisions pour ce faire il faudra envoyer les élève sur tiled pour mettre en place dans leur calque objet des retancgle au quel on ajoutera le type collision.

###### game.py

In [ ]:
import pygame
import pytmx # module pour intégrer la carte
import pyscroll # module pour la gestion de la caméra

from player import Player

class Game:
    def __init__(self):
        # création de la fenêtre
        self.screen = pygame.display.set_mode((800, 600)) # stockage dans une variable
        pygame.display.set_caption("Pygamon - Aventure")
        
        # charger la carte (tmx)
        tmx_data = pytmx.util_pygame.load_pygame('carte.tmx')# Aborder le rangement hiérachique des fichier
        map_data = pyscroll.data.TiledMapData(tmx_data) # traduction en map
        map_layer = pyscroll.orthographic.BufferRenderer(map_data, self.screen.get_size())
        
        # zome sur la carte
        map_layer.zoom = 2 
        
        # generer un joueur
        player_position = tmx_data.get_object_by_name("player")
        self.player = Player(player_position.x , player_position.y)
        
        # stockage des collisions 
        self.wall = []
        
        for obj in tmx_data.objects:
            if obj.type == "collision":
                self.wall.append(pygame.Rect(obj.x, obj.y, obj.width, obj.height))
        
        #dessiner le groupe de calques 
        self.group = pyscroll.PyscrollGroup(map_layer=map_layer, default_layer=3) # calque changement
        self.group.add(self.player)
        
    def handle_input(self):
        pressed = pygame.key.get_pressed()
        
        if pressed[pygame.K_UP]:
            self.player.move_up()
            self.player.change_animation('up')
        elif pressed[pygame.K_DOWN]:
            self.player.move_down()
            self.player.change_animation('down')
        elif pressed[pygame.K_LEFT]:
            self.player.move_left()
            self.player.change_animation('left')
        elif pressed[pygame.K_RIGHT]:
            self.player.move_right()
            self.player.change_animation('right')
    
    def update(self):
        self.group.update()
        
        #verification des collisions
        for sprite in self.group.sprites():
            if sprite.feet.collidelist(self.walls) > -1 :
                sprite.move_back()
    
    def run(self):
        #mise en place des fps
        clock = pygame.time.Clock()
        
        # boucle du jeu
        running = True

        while running :
            
            # sauvegarde de l'ancinne position du joueur 
            self.player.save_location()
            
            # récupération des touches
            self.handle_input()
            
            # mettre à jour la position du joueur et les collisions
            self.update()
            
            # centrer la caméra sur le joueur
            self.group.center(self.player.rect.center)
            
            # affichage de la carte 
            self.group.draw(self.screen)
            
            pygame.display.flip() #mettre à jour tout
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
            
            clock.tick(60)

        pygame.quit()

###### player.py

In [ ]:
import pygame

class Player(pygame.sprite.Sprite):
    def __init__(self, x , y):
        super().__init__()
        self.sprite_sheet = pygame.image.load('player.png')
        self.image = self.get_image(0, 0)
        self.image.set_colorkey([0, 0, 0])
        self.rect = self.image.get_rect()
        self.position = [x, y]
        self.images = {
            'down': self.get_image(0, 0),
            'left': self.get_image(0, 32),
            'right': self.get_image(0, 64,
            'up': self.get_image(0, 96)
        }
        self.feet = pygame.Rect(0, 0, self.rect.width * 0.5, 12)
        self.old_position = self.position.copy()
        self.speed = 3
                                
    def save_location(self): self.old_position = self. position.copy()
                                    
    def change_animation(self, name):                             
        self.image = self.images[name]
        self.image.set_colorkey([0, 0, 0])
        
    def move_right(self): self.position[0] += self.speed
        
    def move_left(self): self.position[0] -= self.speed
        
    def move_up(self): self.position[1] += self.speed
        
    def move_down(self): self.position[1] -= self.speed
        
    def update(self):
        self.rect.topleft = self.position
        self.feet.midbottom = self.rect.midbottom
    
    def move_back(self):
        self.position = self.old_position
        self.rect.topleft = self.position
        self.feet.midbottom = self.rect.midbottom
        
    def get_image(self, x, y):
        image = pygame.Surface([32, 32])
        image.blit(self.sprite_sheet, (0, 0), (x, y, 32, 32))
        return image

### Ordre de passage 
#### 1) ajout des collision
ajouter les collision sur tiled en utilisant un claque objet et des rectangle au quel on attribue le type collision 

#### 2) stockage des collision 
on va stocker toute les collision de la map dans une liste

    class Game:
            def __init__(self):
                # création de la fenêtre
                self.screen = pygame.display.set_mode((800, 600)) 
                pygame.display.set_caption("Pygamon - Aventure")

                # charger la carte (tmx)
                tmx_data = pytmx.util_pygame.load_pygame('carte.tmx')
                map_data = pyscroll.data.TiledMapData(tmx_data) # traduction en map
                map_layer = pyscroll.orthographic.BufferRenderer(map_data, self.screen.get_size())

                # zome sur la carte
                map_layer.zoom = 2 

                # generer un joueur
                player_position = tmx_data.get_object_by_name("player")
                self.player = Player(player_position.x , player_position.y)

    ---->       # stockage des collisions 
    ---->       self.wall = []

    ---->       for obj in tmx_data.objects:
    ---->           if obj.type == "collision":
    ---->               self.wall.append(pygame.Rect(obj.x, obj.y, obj.width, obj.height))

                #dessiner le groupe de calques 
                self.group = pyscroll.PyscrollGroup(map_layer=map_layer, default_layer=3) 
                self.group.add(self.player)
                
#### 3) Mise en place du rectangle de collision du joueur 
on va commencer par lui crée un rectangle de collision puis lui crée multiple fonction pour le faire revenir en arrière lorsqu'il rentre en collision avec un mur 


    class Player(pygame.sprite.Sprite):
            def __init__(self, x , y):
                super().__init__()
                self.sprite_sheet = pygame.image.load('player.png')
                self.image = self.get_image(0, 0)
                self.image.set_colorkey([0, 0, 0])
                self.rect = self.image.get_rect()
                self.position = [x, y]
                self.images = {
                    'down': self.get_image(0, 0),
                    'left': self.get_image(0, 32),
                    'right': self.get_image(0, 64,
                    'up': self.get_image(0, 96)
                }
    ---->       self.feet = pygame.Rect(0, 0, self.rect.width * 0.5, 12)
    ---->       self.old_position = self.position.copy()
                self.speed = 3

    ---->    def save_location(self): self.old_position = self. position.copy()

Donc nous avons aussi crée une variable old position qui sauvegardera l'anciènne position du joueur avant d'avancé. Maintenant il faut mettre tout ça à jour dans update et ajouter une nouvelle fontion move_back

              def update(self):
                  self.rect.topleft = self.position
    ---->         self.feet.midbottom = self.rect.midbottom

    ---->     def move_back(self):
    ---->         self.rect.topleft = self.position
    ---->         self.feet.midbottom = self.rect.midbottom
    
Grace au self.feet.midbottom dans update nous le rattachon à la position du joueur et la fonction move back remet la position comme update sauf que nous devons rajoutée une ligne qui permettra de récupérer l'ancienne position

    self.position = self.old_position
    
Ce qui donne donc :
    
              def move_back(self):
    ---->         self.position = self.old_position
                  self.rect.topleft = self.position
                  self.feet.midbottom = self.rect.midbottom

#### 4) Mise en liaison des différente fonction de collision
On va maintenant tout mettre en place dans game.py pour que le sytème de collision soit fonctionnel.

Pour ce faire on va crée une fonction update qui remplacera la ligne :

    self.group.update()

parce que cette fonction repprendra cette ligne 

         def update(self):
    ---->       self.group.update()

                #verification des collisions
                for sprite in self.group.sprites():
                    if sprite.feet.collidelist(self.walls) > -1 :
                        sprite.move_back()

Une fois la fonction update faite maintont tou en place dans notre boucle principale sans oublier de bien sauvegarder l'anciènne position du joueur avant toute autre action.

    while running :
            
    ---->       # sauvegarde de l'ancinne position du joueur 
    ---->       self.player.save_location()

                # récupération des touches
                self.handle_input()

                # mettre à jour la position du joueur et les collisions
    ---->       self.update()

                # centrer la caméra sur le joueur
                self.group.center(self.player.rect.center)

                # affichage de la carte 
                self.group.draw(self.screen)

                pygame.display.flip() #mettre à jour tout

                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False

                clock.tick(60)
                
__lancer un teste final__

## Ajout de décoration
proposer d'ajouter des décoration à la map et d'y intégrer le système de collision

## Ajout des maison 
Lors de la création du calque tuile ne pas oublier de definir une couche de transparance sinon il y aura un alpha blanc qui restera sur les tuile de maison